In [26]:
import os
import pandas as pd
from decimal import Decimal
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import yfinance as yf


import matplotlib.pyplot as plt
import plotly.express as px
import pytz  # Make sure to import pytz for timezone handling
import seaborn as sns
from sklearn.linear_model import LinearRegression

import requests
import csv
import json

import warnings

In [27]:
load_dotenv()

API_KEY = os.getenv("alpha_vantage_api_key")

In [28]:
# Addtional setting session
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
pd.options.mode.copy_on_write = True

# PE TTM valuation

In [38]:
# Parameters section

alpha_vantage_api_key = API_KEY # FREE TIER API rate limit is 25 requests per day
alpha_vantage_function = {
    'core':[
        'TIME_SERIES_INTRADA'
        ,'TIME_SERIES_DAILY' # this is daily time series quote
        ,'TIME_SERIES_DAILY_ADJUSTED' # this is daily time series adjusted by split/dividend-adjusted
        ,'GLOBAL_QUOTE'
    ]
    ,'fundmental':[
    'INCOME_STATEMENT'
    ,'BALANCE_SHEET' # this is daily time series quote
    ,'CASH_FLOW' # this is daily time series adjusted by split/dividend-adjusted
    ,'EARNINGS'
    ,'EARNINGS_CALENDAR'
]
}

# Custmized
ticker_symbols = [

    'CL'
    ,'ACN'
    ,'IT'
    # ,'GOOG'
    # ,'TSM'
    # ,'META'
    # ,'LPG'
    # ,'NVDA'
    # ,'SKWD'
    # ,'MSFT'
    # ,'NFLX'
    # ,'V'
    # ,'PG'
    # ,'INTU'
    # ,'ISRG'
    ]

# ticker_symbols = [
#  'MO',
#  'ADM',
# #  'BF.B',
#  'BG',
#  'CPB',
#  'CHD',
#  'CLX',
#  'KO',
#  'CL',
#  'CAG',
#  'STZ',
#  'COST',
#  'DG',
#  'DLTR',
#  'EL',
#  'GIS',
#  'HSY',
#  'HRL',
# #  'KVUE',
#  'KDP',
#  'KMB',
#  'KHC',
#  'KR',
#  'LW',
#  'MKC',
#  'TAP',
#  'MDLZ',
#  'MNST',
#  'PEP',
#  'PM',
#  'PG',
#  'SJM',
#  'SYY',
#  'TGT',
#  'TSN',
# #  'WBA',
#  'WMT'
#  ]


# Time intelligent parameters
window_days = 90
end_date = datetime.now()
start_date = end_date - timedelta(days=window_days)
earning_calendar = [
    3  # this will return next 1 qtr forecast earning; nowadays the earning calendar only shows the next 1 qtr forecast earning
    ,6  # this will return next 2 qtr forecast earning
    ,12  # this will return next 4 qtr forecast earning
]

PE_yr_range = 9 # this will return x-1 yr PE range, cuz you cannot get current year end PE

ticker_dict_json = {}
ticker_dict_pd = {}

In [39]:
def convert_to_numeric_or_zero(value):
    try:
        # Try to convert the string to a float (or int)
        num = round(float(value), 2)
        return num  # Return the original value if it can be converted to a number
    except ValueError:
        # If conversion fails, return '0'
        return 0

In [40]:
# Daily quote section
for j, symbol in enumerate(ticker_symbols):

    print(j, symbol)
    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                # Use pandas index assign instead of direct assign value
                stock_split_record_df = pd.DataFrame({
                'split_factor': [1],
                'effective_date': [datetime.today()]
            })
                # stock_split_record_df = pd.DataFrame()
                # stock_split_record_df['split_factor'] = 1
                # stock_split_record_df['effective_date'] = datetime.today()


    # Daily quote section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}&outputsize=full'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Time Series (Daily)':


            selected_cols = [
                '4. close'
            ]

            Daily_stock_df = pd.DataFrame(value).transpose()[selected_cols] # tranpose the dataframe and sub select selected cols

            # Rename columns
            Daily_stock_df.rename(
                columns={
                    '4. close': 'stock_price'
                    }
                ,inplace=True
                )
            
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].round(2)
            Daily_stock_df.index = pd.to_datetime(Daily_stock_df.index)


    for date_i in Daily_stock_df.index.date:
        for date_j in stock_split_record_df['effective_date'].dt.date:
            if date_i == date_j:

                # stock price to divided the split factor
                Daily_stock_df.loc[Daily_stock_df.index.date < date_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.date == date_j].values[0])


    # MA200 calculation
    Daily_stock_df['MA200'] = Daily_stock_df.sort_index(ascending=True)['stock_price'].rolling(window=200).mean()

    # Calculate the slope of the MA200 using linear regression
    def calculate_slope(series):
        # Create a time index (0, 1, 2, ..., n) for the linear regression
        x = np.arange(len(series)).reshape(-1, 1)
        y = series.values.reshape(-1, 1)

        # Fit the linear regression model
        model = LinearRegression()
        model.fit(x, y)

        # Return the slope
        return model.coef_[0][0]


    # Apply the slope calculation to the MA200 values
    Daily_stock_df['MA200_slope'] = Daily_stock_df.sort_index(ascending=True)['MA200'].rolling(window=20).apply(calculate_slope, raw=False)



    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)

    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': 'stock_price'
            }
        ,inplace=True
        )

    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].round(2)

    # modify stock price based on stock split
    for year_i in Monthly_stock_df.index.year:
        for year_j in stock_split_record_df['effective_date'].dt.year:
            if year_i == year_j:

                # stock price to divided the split factor
                Monthly_stock_df.loc[Monthly_stock_df.index.year < year_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.year == year_j].values[0])



    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - PE_yr_range) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue
            
            # clean annualEPS_df
            annualEPS_df = annualEPS_df.sort_values('reportedEPS', ascending=False).drop_duplicates('fiscalDateEnding')
            annualEPS_df = annualEPS_df.sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)

            # calculate metrics
            if Monthly_stock_df.shape[0] <= annualEPS_df.shape[0]: # if Monthly_stock_df has less records than annualEPS_df, choose the mini length record
                annualEPS_df = annualEPS_df[:(Monthly_stock_df.shape[0])]

                annualEPS_df["PE"] = Monthly_stock_df["stock_price"].values / annualEPS_df['reportedEPS'].values
                annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] = annualEPS_df["PE"].mean().round(2)
                annualEPS_df[f"PE_{PE_yr_range-1}yr_std"] = np.std(annualEPS_df["PE"]).round(2)
                annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_+"] = (annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] + annualEPS_df[f"PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围上限
                annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_-"] = (annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] - annualEPS_df[f"PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围下限



        if key == 'quarterlyEarnings':

            selected_cols = [
                'reportedDate'
                ,'reportedEPS'
            ]

            qtrEPS_df = pd.DataFrame(value)[selected_cols] # tranpose the dataframe and sub select selected cols
            qtrEPS_df['reportedDate'] = pd.to_datetime(qtrEPS_df['reportedDate'])

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    qtrEPS_df[col] = qtrEPS_df[col].astype(str).apply(lambda x: float(x) if x not in [None, 'None', 'nan', 'NaN'] else float(0))
                else:
                    continue


    # # forecast 1 qtr earnings from alpha vantage API -- for now we use yf API
    # # for i in earning_calendar: comment out the for loop in case of future usage, i can be the parameter of {}month
    # CSV_URL = f'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol={symbol}&horizon=12month&apikey={alpha_vantage_api_key}'
    # with requests.Session() as s:
    #     download = s.get(CSV_URL)
    #     decoded_content = download.content.decode('utf-8')
    #     cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    #     my_list = list(cr)

    #     forecast_earanings_df = pd.DataFrame(
    #         columns=my_list[0]
    #         ,data=my_list[1::]
    #         )
        
    #     if forecast_earanings_df['estimate'].head(1).values != '':
    #         latest_projected_EPS = float(forecast_earanings_df['estimate'].head(1).values)
    #     else:
    #         latest_projected_EPS = 0


    # forecast 1 qtr earning, 1yr earning from yf API
    yf_symbol = yf.Ticker(symbol)
    stock_eps_forecast_df = yf_symbol.earnings_estimate
    stock_eps_forecast_df['period'] = stock_eps_forecast_df.index
    next_qtr_EPS = stock_eps_forecast_df[stock_eps_forecast_df['period'] == '+1q']['avg'].values[0]
    # next yr eps
    next_yr_EPS = stock_eps_forecast_df[stock_eps_forecast_df['period'] == '+1y']['avg'].values[0]


    # EPS trend from yf API
    stock_eps_trend_df = yf_symbol.eps_trend
    stock_eps_trend_df['period'] = stock_eps_trend_df.index
    next_yr_days7ago_EPS = stock_eps_trend_df[stock_eps_trend_df['period'] == '+1y']['7daysAgo'].values[0]
    next_yr_days30ago_EPS = stock_eps_trend_df[stock_eps_trend_df['period'] == '+1y']['30daysAgo'].values[0]
    next_yr_days60ago_EPS = stock_eps_trend_df[stock_eps_trend_df['period'] == '+1y']['60daysAgo'].values[0]
    next_yr_days90ago_EPS = stock_eps_trend_df[stock_eps_trend_df['period'] == '+1y']['90daysAgo'].values[0]



    # stock growth rate from yf API
    stock_growth_est_df = yf_symbol.growth_estimates
    stock_growth_est_df['period'] = stock_growth_est_df.index
    curr_yr_growthrate_symbol = stock_growth_est_df[stock_growth_est_df['period'] == '0y']['stockTrend'].values[0] * 100
    next_yr_growthrate_symbol = stock_growth_est_df[stock_growth_est_df['period'] == '+1y']['stockTrend'].values[0] * 100
    curr_yr_growthrate_index = stock_growth_est_df[stock_growth_est_df['period'] == '0y']['indexTrend'].values[0] * 100
    next_yr_growthrate_index = stock_growth_est_df[stock_growth_est_df['period'] == '+1y']['indexTrend'].values[0] * 100


    # US Treasury section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=10year&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            US_T_10yrs_df = pd.DataFrame(value)
            US_T_10yrs_df['value'] = pd.to_numeric(US_T_10yrs_df['value'], errors='coerce') # change dataframe value to numeric data
            US_T_10yrs_YTM = US_T_10yrs_df['value'][0]



    # COMPANY OVERVIEW
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    
    for key, value in data.items():
        if key  == 'MarketCapitalization':
            stock_mkt_cap = float(value)



    # INCOME STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()


    for key, value in data.items():
        if key == 'annualReports':
            annual_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_income_df = annual_income_df.sort_values('fiscalDateEnding', ascending=True)

            # annual measurements cols
            annual_income_metric_cols = annual_income_df.columns[1:]

            # Calculate quarter-over-quarter change percentage
            for column in annual_income_metric_cols:  # Exclude the 'fiscalDateEnding' column
                annual_income_df[column] = pd.to_numeric(annual_income_df[column], errors='coerce')
                annual_income_df[f'{column}_YoY'] = annual_income_df[column].pct_change() * 100 
            
            annual_income_YoY_metric_cols = [col for col in annual_income_df.columns if 'YoY' in col]

            # ratios of income statement calculation
            annual_income_df['gross_margin_%'] = (annual_income_df['grossProfit'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['operating_margin_%'] = (annual_income_df['operatingIncome'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['net_margin_%'] = (annual_income_df['netIncome'] / annual_income_df['totalRevenue']) * 100



        if key == 'quarterlyReports':
            qtr_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_income_df = qtr_income_df.sort_values('fiscalDateEnding', ascending=True)

            # qtr measurements cols
            qtr_income_metric_cols = qtr_income_df.columns[1:]
            
            # Calculate quarter-over-quarter change percentage
            for column in qtr_income_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_income_df[column] = pd.to_numeric(qtr_income_df[column], errors='coerce')
                qtr_income_df[f'{column}_QoQ'] = qtr_income_df[column].pct_change() * 100

            qtr_income_QoQ_metric_cols = [col for col in qtr_income_df.columns if 'QoQ' in col]

            # ratios of income statement calculation
            qtr_income_df['gross_margin_%'] = (qtr_income_df['grossProfit'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['operating_margin_%'] = (qtr_income_df['operatingIncome'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['net_margin_%'] = (qtr_income_df['netIncome'] / qtr_income_df['totalRevenue']) * 100

    income_ratio_cols = [
        'gross_margin_%'
        ,'operating_margin_%'
        ,'net_margin_%'
        ,'netIncome'
        ]
    
    

    # BALANCESHEET
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_balancesheet_df = annual_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_balancesheet_df[column] = pd.to_numeric(annual_balancesheet_df[column], errors='coerce')

            annual_balancesheet_df['current_ratio'] = (annual_balancesheet_df['totalCurrentAssets'] / annual_balancesheet_df['totalCurrentLiabilities'])
            annual_balancesheet_df['working_capital'] = annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['totalCurrentLiabilities']
            annual_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (annual_balancesheet_df['longTermDebt'] / annual_balancesheet_df['working_capital'])
            annual_balancesheet_df['debtEquity_ratio'] = (annual_balancesheet_df['totalLiabilities'] / annual_balancesheet_df['totalShareholderEquity'])
            annual_balancesheet_df['quick_ratio'] = ((annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['inventory']) / annual_balancesheet_df['totalCurrentLiabilities'])

            
        if key == 'quarterlyReports':
            qtr_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_balancesheet_df = qtr_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_balancesheet_df[column] = pd.to_numeric(qtr_balancesheet_df[column], errors='coerce')

    
            qtr_balancesheet_df['current_ratio'] = (qtr_balancesheet_df['totalCurrentAssets'] / qtr_balancesheet_df['totalCurrentLiabilities'])
            qtr_balancesheet_df['working_capital'] = qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['totalCurrentLiabilities']
            qtr_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (qtr_balancesheet_df['longTermDebt'] / qtr_balancesheet_df['working_capital'])
            qtr_balancesheet_df['debtEquity_ratio'] = (qtr_balancesheet_df['totalLiabilities'] / qtr_balancesheet_df['totalShareholderEquity'])
            qtr_balancesheet_df['quick_ratio'] = ((qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['inventory']) / qtr_balancesheet_df['totalCurrentLiabilities'])

            qtr_balancesheet_df['BVPS_latest'] = round(
                qtr_balancesheet_df.tail(1)['totalShareholderEquity'].sum() / qtr_balancesheet_df.tail(1)['commonStockSharesOutstanding']
                ,2
                )



    balancesheet_ratio_cols = [
        'current_ratio'
        ,'working_capital'
        ,'longTermDebt_to_workingCp_ratio'
        ,'debtEquity_ratio'
        ,'quick_ratio'
        ,'totalShareholderEquity'
    ]


    # CASHFLOW STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_cashflow_df = annual_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_cashflow_df[column] = pd.to_numeric(annual_cashflow_df[column], errors='coerce')
                annual_cashflow_df[f'{column}_YoY'] = annual_cashflow_df[column].pct_change() * 100

            
        if key == 'quarterlyReports':
            qtr_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_cashflow_df = qtr_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_cashflow_df[column] = pd.to_numeric(qtr_cashflow_df[column], errors='coerce')
                qtr_cashflow_df[f'{column}_YoY'] = qtr_cashflow_df[column].pct_change() * 100


    stock_ratios_annual_consolidate_df = pd.DataFrame()
    stock_ratios_qtr_consolidate_df = pd.DataFrame()

    stock_ratios_annual_consolidate_df[income_ratio_cols] = annual_income_df[income_ratio_cols]
    stock_ratios_annual_consolidate_df[balancesheet_ratio_cols] = annual_balancesheet_df[balancesheet_ratio_cols]

    stock_ratios_qtr_consolidate_df[income_ratio_cols] = qtr_income_df[income_ratio_cols]
    stock_ratios_qtr_consolidate_df[balancesheet_ratio_cols] = qtr_balancesheet_df[balancesheet_ratio_cols]

    # calculating new consolidated metrics 
    # annual df
    stock_ratios_annual_consolidate_df['ROE_%'] = (annual_income_df['netIncome'] / annual_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_annual_consolidate_df['liquidation_value'] = ((annual_balancesheet_df['totalAssets'] - annual_balancesheet_df['intangibleAssets']) - annual_balancesheet_df['totalLiabilities'])
    # stock_ratios_annual_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_annual_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_annual_consolidate_df['fiscalDateEnding'] = annual_balancesheet_df['fiscalDateEnding']

    # qtr df
    stock_ratios_qtr_consolidate_df['ROE_%'] = (qtr_income_df['netIncome'] / qtr_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_qtr_consolidate_df['liquidation_value'] = ((qtr_balancesheet_df['totalAssets'] - qtr_balancesheet_df['intangibleAssets']) - qtr_balancesheet_df['totalLiabilities'])
    # stock_ratios_qtr_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_qtr_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_qtr_consolidate_df['fiscalDateEnding'] = qtr_balancesheet_df['fiscalDateEnding']





    # Consolidated section
    stock_consolidate_df = Daily_stock_df.head(window_days)


    stock_consolidate_df_date = stock_consolidate_df.index
    for i in stock_consolidate_df_date:
                
        # Filter the DataFrame to include only dates(index) less than or equal to the target date
        filtered_qtrEPS_df = qtrEPS_df[qtrEPS_df['reportedDate'] < i]

        # Select the first four rows from the past_qtrs_EPS
        past_4_qtrs_EPS = filtered_qtrEPS_df.head(4) 
        past_3_qtrs_EPS = filtered_qtrEPS_df.head(3)
        past_1_qtr_EPS = filtered_qtrEPS_df.head(1)

        # Calculate the sum of the numeric values in the selected rows
        EPS_TTM = past_4_qtrs_EPS['reportedEPS'].values.sum()
        EPS_curr_qtr = past_1_qtr_EPS['reportedEPS'].values.sum()

        # assign each index row with the EPS_TTM
        stock_consolidate_df.loc[i, "EPS_TTM"] = EPS_TTM
        stock_consolidate_df.loc[i, "EPS_currentQtr"] = EPS_curr_qtr

        if i == max(stock_consolidate_df.index):
            EPS_nextQtr_projected = next_qtr_EPS + past_3_qtrs_EPS['reportedEPS'].values.sum()  # This metrics is the past 3 qtrs post EPS + 1 projected EPS
        else:
            continue

        stock_consolidate_df["EPS_nextQtr"] = next_qtr_EPS
        stock_consolidate_df["EPS_nextQtr_TTM"] = EPS_nextQtr_projected
        stock_consolidate_df["EPS_nextYr"] = next_yr_EPS

    

    # stock's stats
    stock_consolidate_df["Ticker"] = symbol
    # stock_consolidate_df["PE_1yr_forward"] = round(PE_12month_projected, 2)
    stock_consolidate_df["PE_TTM"] = (stock_consolidate_df["stock_price"] / stock_consolidate_df["EPS_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_avg"] = stock_consolidate_df["PE_TTM"].mean().round(2)
    stock_consolidate_df["PE_TTM_std"] = np.std(stock_consolidate_df["PE_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_volatility_+"] = (stock_consolidate_df["PE_TTM_avg"] + stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围上限
    stock_consolidate_df["PE_TTM_volatility_-"] = (stock_consolidate_df["PE_TTM_avg"] - stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围下限

    stock_consolidate_df["relative_valuation_TTM_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_TTM_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_TTM_median"] = (np.median([stock_consolidate_df["relative_valuation_TTM_+"][0], stock_consolidate_df["relative_valuation_TTM_-"][0]])).round(2) #这个是根据最新TTM PE估值的价格中位数

    stock_consolidate_df["relative_valuation_nextQuater_projected_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * stock_consolidate_df["EPS_nextQtr_TTM"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_nextQuater_projected_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * stock_consolidate_df["EPS_nextQtr_TTM"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_nextQuater_projected_median"] = (np.median([stock_consolidate_df["relative_valuation_nextQuater_projected_+"][0], stock_consolidate_df["relative_valuation_nextQuater_projected_-"][0]])).round(2) #这个是根据3 qtrs post EPS + 1 projected EPS 得出PE估值的价格中位数

    stock_consolidate_df[f"{window_days}_price_min"] = stock_consolidate_df["stock_price"].min().round(2)
    stock_consolidate_df[f"{window_days}_price_max"] = stock_consolidate_df["stock_price"].max().round(2)
    stock_consolidate_df[f"{window_days}_price_avg"] = stock_consolidate_df["stock_price"].mean().round(2)
    stock_consolidate_df[f"{window_days}_price_std"] = np.std(stock_consolidate_df["stock_price"]).round(2)

    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_avg"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_std"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_std"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_volatility_+"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_+"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_volatility_-"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_-"].values[0]


    stock_consolidate_df["relative_valuation_nextYear_projected_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * next_yr_EPS).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_nextYear_projected_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * next_yr_EPS).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_nextYear_projected_median"] = (np.median([stock_consolidate_df["relative_valuation_nextYear_projected_+"][0], stock_consolidate_df["relative_valuation_nextYear_projected_-"][0]])).round(2) #这个是根据next year projected EPS 得出PE估值的价格中位数

    # stock_consolidate_df["PEG_next12months"] = PEG_12month_projected
    stock_consolidate_df["EPS_nextYr"] = round(next_yr_EPS, 2)
    # stock_consolidate_df["PEG_TTM"] = (stock_consolidate_df["PE_TTM"] / (((next_yr_EPS - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100)).round(2) # 这个是截止下一年的EPS growth rate所得出的PEG ratio, <1是undervalue的表现
    
    stock_consolidate_df["EPS_nextYr_growthRate"] = (((next_yr_EPS - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100).round(2)
    stock_consolidate_df["EPS_nextQtr_growthRate"] = (((stock_consolidate_df["EPS_nextQtr_TTM"] - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100).round(2)

    stock_consolidate_df["EarningYield_TTM"] = ((stock_consolidate_df["EPS_TTM"] / stock_consolidate_df["stock_price"]) * 100).round(2)
    stock_consolidate_df["ERP_TTM"] = stock_consolidate_df["EarningYield_TTM"] - US_T_10yrs_YTM # it means a comparison between equity return and 10 years risk free, usually ERP >= 3 for short term invest at least, ERP >= 5 for long term invest 

    # stock_consolidate_df['latest_qtr_liquidation_mktcap_ratio_%'] = stock_ratios_qtr_consolidate_df['liquidation_mktcap_ratio_%'].values[-1]
    stock_consolidate_df['BVPS_latest'] = qtr_balancesheet_df['BVPS_latest'].values[-1]

    stock_consolidate_df['MA200_slope'] = Daily_stock_df['MA200_slope']

    stock_consolidate_df['FCF_per_share_TTM'] = round((qtr_cashflow_df['operatingCashflow'] - qtr_cashflow_df['capitalExpenditures']).tail(4).sum() / qtr_balancesheet_df['commonStockSharesOutstanding'].values[-1], 2)
    stock_consolidate_df['PFCF_TTM'] = round(stock_consolidate_df['stock_price'][0]  / stock_consolidate_df['FCF_per_share_TTM'], 2)
    stock_consolidate_df['FCF_yield_TTM'] = round((stock_consolidate_df['FCF_per_share_TTM'] / stock_consolidate_df['stock_price'][0]) * 100, 2) # % return on cash of every single dollar you spent on stock price, the higher the better

    # EPS trend for next year estimates
    stock_consolidate_df['next_yr_days7ago_EPS'] = round(next_yr_days7ago_EPS, 2)
    stock_consolidate_df['next_yr_days30ago_EPS'] = round(next_yr_days30ago_EPS, 2)
    stock_consolidate_df['next_yr_days60ago_EPS'] = round(next_yr_days60ago_EPS, 2)
    stock_consolidate_df['next_yr_days90ago_EPS'] = round(next_yr_days90ago_EPS, 2)

    # symbol growth rate
    stock_consolidate_df['curr_yr_growthrate_symbol'] = round(curr_yr_growthrate_symbol, 2)
    stock_consolidate_df['next_yr_growthrate_symbol'] = round(next_yr_growthrate_symbol, 2)
    stock_consolidate_df['curr_yr_growthrate_index'] = round(curr_yr_growthrate_index, 2)
    stock_consolidate_df['next_yr_growthrate_index'] = round(next_yr_growthrate_index, 2)

    # filter conditions
    conditions = [
    (stock_consolidate_df["stock_price"] < stock_consolidate_df["relative_valuation_TTM_-"]),
    (stock_consolidate_df["stock_price"] > stock_consolidate_df["relative_valuation_TTM_+"]),
    ((stock_consolidate_df["stock_price"] >= stock_consolidate_df["relative_valuation_TTM_-"]) & (stock_consolidate_df["stock_price"] <= stock_consolidate_df["relative_valuation_TTM_+"])),
    ]

    categories = [
        'undervalued'
        ,'overvalued'
        ,'fair'
        ]

    # This KPI assess if the current stock price is under/over/fair to the current relative valuation
    stock_consolidate_df["curr_assessment"] = None

    for condition, category in zip(conditions, categories):
        stock_consolidate_df.loc[condition, "price_valuation_assessment"] = category





    # Append key-value pairs to the dictionary
    selected_cols = [
    "Ticker"
    ,"stock_price"
    ,"EPS_currentQtr"
    ,"EPS_nextQtr"
    ,"EPS_TTM"
    ,"EPS_nextQtr_TTM"
    ,"EPS_nextYr"
    # ,"PE_1yr_forward"
    ,"PE_TTM"
    ,"PE_TTM_avg"
    ,"PE_TTM_volatility_+"
    ,"PE_TTM_volatility_-"
    ,f"PE_{PE_yr_range-1}yr_avg"
    ,f"PE_{PE_yr_range-1}yr_volatility_+"
    ,f"PE_{PE_yr_range-1}yr_volatility_-"
    ,"relative_valuation_TTM_+"
    ,"relative_valuation_TTM_-"
    ,"relative_valuation_TTM_median"
    ,"relative_valuation_nextQuater_projected_+"
    ,"relative_valuation_nextQuater_projected_-"
    ,"relative_valuation_nextQuater_projected_median"
    ,"relative_valuation_nextYear_projected_+"
    ,"relative_valuation_nextYear_projected_-"
    ,"relative_valuation_nextYear_projected_median"
    ,"price_valuation_assessment"
    ,"EPS_nextQtr_growthRate"
    ,"EPS_nextYr_growthRate"
    # ,"PEG_next12months"
    # ,"PEG_TTM"
    ,"EarningYield_TTM"
    ,"ERP_TTM"
    # ,"latest_qtr_liquidation_mktcap_ratio_%"
    ,"BVPS_latest"
    ,"PFCF_TTM"
    ,"FCF_yield_TTM" 
    # ,"MA200_slope"
    ,'next_yr_days7ago_EPS'
    ,'next_yr_days30ago_EPS'
    ,'next_yr_days60ago_EPS'
    ,'next_yr_days90ago_EPS'
    ,'curr_yr_growthrate_symbol'
    ,'next_yr_growthrate_symbol'
    ,'curr_yr_growthrate_index'
    ,'next_yr_growthrate_index'
    ]


    # store each stock info as pd into dictionary
    ticker_dict_pd[symbol] = stock_consolidate_df[selected_cols]
    # transfer pandas dataframe to json format, and each stock info into dictionary
    ticker_dict_json[symbol] = stock_consolidate_df[selected_cols].to_dict()

    if j == 0:
        # screener df creation
        # screener df will store each stock's consolidate df's first row and union them together for screening purposee
        ticker_screen_df = pd.DataFrame(
            columns=selected_cols
        ) 
        stock_consolidate_df_values = stock_consolidate_df[selected_cols].values[0]

        # Insert rows into the DataFrame
        ticker_screen_df.loc[j] = stock_consolidate_df_values

    else:
        stock_consolidate_df_values = stock_consolidate_df[selected_cols].values[0]

        
        ticker_screen_df.loc[j] = stock_consolidate_df_values

# ticker screen df consolidated metrics
ticker_screen_df['Industry_PE_TTM_avg'] = round(ticker_screen_df['PE_TTM'].mean(), 2)

0 CL
1 ACN
2 IT


In [41]:
ticker_dict_pd['IT']

,Ticker,stock_price,EPS_currentQtr,EPS_nextQtr,EPS_TTM,EPS_nextQtr_TTM,EPS_nextYr,PE_TTM,PE_TTM_avg,PE_TTM_volatility_+,PE_TTM_volatility_-,PE_8yr_avg,PE_8yr_volatility_+,PE_8yr_volatility_-,relative_valuation_TTM_+,relative_valuation_TTM_-,relative_valuation_TTM_median,relative_valuation_nextQuater_projected_+,relative_valuation_nextQuater_projected_-,relative_valuation_nextQuater_projected_median,relative_valuation_nextYear_projected_+,relative_valuation_nextYear_projected_-,relative_valuation_nextYear_projected_median,price_valuation_assessment,EPS_nextQtr_growthRate,EPS_nextYr_growthRate,EarningYield_TTM,ERP_TTM,BVPS_latest,PFCF_TTM,FCF_yield_TTM,next_yr_days7ago_EPS,next_yr_days30ago_EPS,next_yr_days60ago_EPS,next_yr_days90ago_EPS,curr_yr_growthrate_symbol,next_yr_growthrate_symbol,curr_yr_growthrate_index,next_yr_growthrate_index
2026-02-19,IT,156.94,3.36,3.6897,12.36,13.3397,14.51,12.70,15.74,17.33,14.15,33.15,38.92,27.38,214.20,174.89,194.54,231.18,188.76,209.97,251.45,205.31,228.38,undervalued,7.93,17.39,7.88,3.79,4.44,9.63,10.39,14.49,14.88,14.91,14.96,-0.03,10.2,14.19,15.69
2026-02-18,IT,161.25,3.36,3.6897,12.36,13.3397,14.51,13.05,15.74,17.33,14.15,33.15,38.92,27.38,214.20,174.89,194.54,231.18,188.76,209.97,251.45,205.31,228.38,undervalued,7.93,17.39,7.67,3.58,4.44,9.63,10.39,14.49,14.88,14.91,14.96,-0.03,10.2,14.19,15.69
2026-02-17,IT,156.30,3.36,3.6897,12.36,13.3397,14.51,12.65,15.74,17.33,14.15,33.15,38.92,27.38,214.20,174.89,194.54,231.18,188.76,209.97,251.45,205.31,228.38,undervalued,7.93,17.39,7.91,3.82,4.44,9.63,10.39,14.49,14.88,14.91,14.96,-0.03,10.2,14.19,15.69
2026-02-13,IT,158.58,3.36,3.6897,12.36,13.3397,14.51,12.83,15.74,17.33,14.15,33.15,38.92,27.38,214.20,174.89,194.54,231.18,188.76,209.97,251.45,205.31,228.38,undervalued,7.93,17.39,7.79,3.70,4.44,9.63,10.39,14.49,14.88,14.91,14.96,-0.03,10.2,14.19,15.69
2026-02-12,IT,153.63,3.36,3.6897,12.36,13.3397,14.51,12.43,15.74,17.33,14.15,33.15,38.92,27.38,214.20,174.89,194.54,231.18,188.76,209.97,251.45,205.31,228.38,undervalued,7.93,17.39,8.05,3.96,4.44,9.63,10.39,14.49,14.88,14.91,14.96,-0.03,10.2,14.19,15.69
2026-02-11,IT,161.53,3.36,3.6897,12.36,13.3397,14.51,13.07,15.74,17.33,14.15,33.15,38.92,27.38,214.20,174.89,194.54,231.18,188.76,209.97,251.45,205.31,228.38,undervalued,7.93,17.39,7.65,3.56,4.44,9.63,10.39,14.49,14.88,14.91,14.96,-0.03,10.2,14.19,15.69
2026-02-10,IT,159.89,3.36,3.6897,12.36,13.3397,14.51,12.94,15.74,17.33,14.15,33.15,38.92,27.38,214.20,174.89,194.54,231.18,188.76,209.97,251.45,205.31,228.38,undervalued,7.93,17.39,7.73,3.64,4.44,9.63,10.39,14.49,14.88,14.91,14.96,-0.03,10.2,14.19,15.69
2026-02-09,IT,159.75,3.36,3.6897,12.36,13.3397,14.51,12.92,15.74,17.33,14.15,33.15,38.92,27.38,214.20,174.89,194.54,231.18,188.76,209.97,251.45,205.31,228.38,undervalued,7.93,17.39,7.74,3.65,4.44,9.63,10.39,14.49,14.88,14.91,14.96,-0.03,10.2,14.19,15.69
2026-02-06,IT,156.33,3.36,3.6897,12.36,13.3397,14.51,12.65,15.74,17.33,14.15,33.15,38.92,27.38,214.20,174.89,194.54,231.18,188.76,209.97,251.45,205.31,228.38,undervalued,7.93,17.39,7.91,3.82,4.44,9.63,10.39,14.49,14.88,14.91,14.96,-0.03,10.2,14.19,15.69
2026-02-05,IT,152.03,3.36,3.6897,12.36,13.3397,14.51,12.30,15.74,17.33,14.15,33.15,38.92,27.38,214.20,174.89,194.54,231.18,188.76,209.97,251.45,205.31,228.38,undervalued,7.93,17.39,8.13,4.04,4.44,9.63,10.39,14.49,14.88,14.91,14.96,-0.03,10.2,14.19,15.69


# Stock screener

In [54]:
ticker_screen_df = ticker_screen_df.sort_values(by=['price_valuation_assessment']
                                                ,ascending=False)

selected_cols = [
    'Ticker'

    ,'BVPS_latest'
    ,'stock_price'
    # Earnings
    ,'EPS_TTM'
    ,"EPS_nextYr"

    # PE relative valuation
    ,'PE_TTM'
    ,'PE_TTM_avg'
    ,f"PE_{PE_yr_range-1}yr_avg"
    ,f"PE_{PE_yr_range-1}yr_volatility_+"
    ,f"PE_{PE_yr_range-1}yr_volatility_-"
    ,"relative_valuation_TTM_+"
    ,"relative_valuation_TTM_-"
    ,"relative_valuation_TTM_median"
    ,"relative_valuation_nextQuater_projected_+"
    ,"relative_valuation_nextQuater_projected_-"
    ,"relative_valuation_nextQuater_projected_median"
    ,"relative_valuation_nextYear_projected_+"
    ,"relative_valuation_nextYear_projected_-"
    ,"relative_valuation_nextYear_projected_median"
    ,'price_valuation_assessment'

    # Growth
    ,"EPS_nextQtr_growthRate"
    ,"EPS_nextYr_growthRate"
    # ,"PEG_next12months"
    # ,"PEG_TTM"

    # Other valuation metrics
    ,'ERP_TTM'
    # ,'latest_qtr_liquidation_mktcap_ratio_%'
    ,'PFCF_TTM'
    ,'FCF_yield_TTM'
    ,'next_yr_days7ago_EPS'
    ,'next_yr_days30ago_EPS'
    ,'next_yr_days60ago_EPS'
    ,'next_yr_days90ago_EPS'
    ,'curr_yr_growthrate_symbol'
    ,'next_yr_growthrate_symbol'
    ,'curr_yr_growthrate_index'
    ,'next_yr_growthrate_index'
    ,'Industry_PE_TTM_avg'
]

ticker_screen_df[selected_cols]

,Ticker,BVPS_latest,stock_price,EPS_TTM,EPS_nextYr,PE_TTM,PE_TTM_avg,PE_8yr_avg,PE_8yr_volatility_+,PE_8yr_volatility_-,relative_valuation_TTM_+,relative_valuation_TTM_-,relative_valuation_TTM_median,relative_valuation_nextQuater_projected_+,relative_valuation_nextQuater_projected_-,relative_valuation_nextQuater_projected_median,relative_valuation_nextYear_projected_+,relative_valuation_nextYear_projected_-,relative_valuation_nextYear_projected_median,price_valuation_assessment,EPS_nextQtr_growthRate,EPS_nextYr_growthRate,ERP_TTM,PFCF_TTM,FCF_yield_TTM,next_yr_days7ago_EPS,next_yr_days30ago_EPS,next_yr_days60ago_EPS,next_yr_days90ago_EPS,curr_yr_growthrate_symbol,next_yr_growthrate_symbol,curr_yr_growthrate_index,next_yr_growthrate_index,Industry_PE_TTM_avg
13,EL,11.05,99.47,1.95,3.01,51.01,61.92,44.65,59.25,30.05,131.78,109.71,120.74,111.88,93.14,102.51,203.3,169.25,186.28,undervalued,-15.1,54.27,-2.25,31.88,3.14,2.94,2.93,2.92,2.92,45.58,36.85,16.1,15.41,21.88
0,MO,-2.09,65.4,5.42,5.8,12.07,11.23,11.49,15.08,7.9,64.39,57.34,60.86,67.26,59.9,63.58,68.93,61.39,65.16,overvalued,4.45,7.06,4.08,12.09,8.27,5.8,5.79,5.79,5.8,3.12,3.82,16.1,15.41,21.88
15,HSY,22.81,231.53,6.31,9.57,36.69,24.79,23.77,27.35,20.19,170.24,142.61,156.43,156.93,131.46,144.2,258.29,216.36,237.32,overvalued,-7.82,51.72,-1.48,30.38,3.29,8.38,8.19,7.95,8.05,31.65,15.24,16.1,15.41,21.88
32,TSN,52.24,65.26,3.95,4.44,16.52,14.06,15.84,25.22,6.46,58.93,52.14,55.54,60.22,53.27,56.74,66.19,58.56,62.38,overvalued,2.17,12.31,1.84,20.33,4.92,4.41,4.34,4.44,4.65,-4.77,13.07,16.1,15.41,21.88
31,TGT,34.02,115.55,7.54,7.68,15.32,12.73,18.19,24.39,11.99,103.98,87.99,95.98,90.99,77.0,84.0,105.95,89.66,97.8,overvalued,-12.49,1.9,2.32,17.51,5.71,7.67,7.68,7.68,7.92,-17.63,5.28,16.1,15.41,21.88
30,SYY,4.75,86.53,4.58,5.09,18.89,17.03,26.4,37.94,14.86,81.57,74.42,78.0,91.51,83.5,87.5,90.69,82.75,86.72,overvalued,12.19,11.18,1.08,21.58,4.63,5.1,5.05,5.05,5.05,2.96,10.89,16.1,15.41,21.88
29,SJM,56.69,109.51,8.92,7.97,12.28,11.2,13.46,15.89,11.03,103.2,96.6,99.9,104.55,97.86,101.2,92.19,86.29,89.24,overvalued,1.3,-10.67,3.94,18.44,5.42,8.04,8.0,7.78,7.76,-11.19,9.92,16.1,15.41,21.88
28,PG,22.0,159.17,6.79,7.32,23.44,21.5,25.14,27.65,22.63,150.74,141.23,145.99,150.5,141.01,145.76,162.47,152.22,157.34,overvalued,-0.16,7.78,0.06,25.97,3.85,7.31,7.35,7.39,7.39,2.03,5.02,16.1,15.41,21.88
27,PM,-5.34,182.81,7.39,9.11,24.74,21.75,17.31,20.08,14.54,170.04,151.42,160.73,183.44,163.35,173.4,209.58,186.63,198.1,overvalued,7.88,23.25,-0.17,27.16,3.68,9.09,9.11,9.13,9.14,10.73,9.09,16.1,15.41,21.88
26,PEP,14.88,170.49,8.15,9.11,20.92,18.73,23.0,26.57,19.43,157.95,147.35,152.65,173.17,161.55,167.36,176.53,164.69,170.61,overvalued,9.64,11.77,0.57,22.67,4.41,8.99,8.97,8.97,8.98,5.87,5.7,16.1,15.41,21.88


# Price EPS chart

In [45]:
# Parameters section
alpha_vantage_api_key = API_KEY


ticker_symbols = [
    'ACN'
]



In [46]:
# Daily quote section
for j, symbol in enumerate(ticker_symbols):

    print(j, symbol)
    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                # Use pandas index assign instead of direct assign value
                stock_split_record_df = pd.DataFrame({
                'split_factor': [1],
                'effective_date': [datetime.today()]
            })


    # Daily quote section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}&outputsize=full'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Time Series (Daily)':


            selected_cols = [
                '4. close'
            ]

            Daily_stock_df = pd.DataFrame(value).transpose()[selected_cols] # tranpose the dataframe and sub select selected cols

            # Rename columns
            Daily_stock_df.rename(
                columns={
                    '4. close': 'stock_price'
                    }
                ,inplace=True
                )
            
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].round(2)
            Daily_stock_df.index = pd.to_datetime(Daily_stock_df.index)


    for date_i in Daily_stock_df.index.date:
        for date_j in stock_split_record_df['effective_date'].dt.date:
            if date_i == date_j:

                # stock price to divided the split factor
                Daily_stock_df.loc[Daily_stock_df.index.date < date_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.date == date_j].values[0])




    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year


            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue
            
            # clean annualEPS_df
            annualEPS_df = annualEPS_df.sort_values('reportedEPS', ascending=False).drop_duplicates('fiscalDateEnding')
            annualEPS_df = annualEPS_df.sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)



        if key == 'quarterlyEarnings':

            selected_cols = [
                'reportedDate'
                ,'reportedEPS'
            ]

            qtrEPS_df = pd.DataFrame(value)[selected_cols] # tranpose the dataframe and sub select selected cols
            qtrEPS_df['reportedDate'] = pd.to_datetime(qtrEPS_df['reportedDate'])

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    qtrEPS_df[col] = qtrEPS_df[col].astype(str).apply(lambda x: float(x) if x not in [None, 'None', 'nan', 'NaN'] else float(0))
                else:
                    continue



    # Consolidated section
    stock_consolidate_df = Daily_stock_df
    stock_consolidate_df_date = stock_consolidate_df.index
    for i in stock_consolidate_df_date:
                
        # Filter the DataFrame to include only dates(index) less than or equal to the target date
        filtered_qtrEPS_df = qtrEPS_df[qtrEPS_df['reportedDate'] < i]

        # Select the first four rows from the past_qtrs_EPS
        past_4_qtrs_EPS = filtered_qtrEPS_df.head(4) 
        past_3_qtrs_EPS = filtered_qtrEPS_df.head(3)
        past_1_qtr_EPS = filtered_qtrEPS_df.head(1)

        # Calculate the sum of the numeric values in the selected rows
        EPS_TTM = past_4_qtrs_EPS['reportedEPS'].values.sum()
        EPS_curr_qtr = past_1_qtr_EPS['reportedEPS'].values.sum()

        # assign each index row with the EPS_TTM
        stock_consolidate_df.loc[i, "EPS_TTM"] = EPS_TTM
        stock_consolidate_df.loc[i, "EPS_currentQtr"] = EPS_curr_qtr


    # stock's stats
    stock_consolidate_df["Ticker"] = symbol
    stock_consolidate_df["PE_TTM"] = (stock_consolidate_df["stock_price"] / stock_consolidate_df["EPS_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_avg"] = stock_consolidate_df["PE_TTM"].mean().round(2)
    stock_consolidate_df["PE_TTM_std"] = np.std(stock_consolidate_df["PE_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_volatility_+"] = (stock_consolidate_df["PE_TTM_avg"] + stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围上限
    stock_consolidate_df["PE_TTM_volatility_-"] = (stock_consolidate_df["PE_TTM_avg"] - stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围下限

    stock_consolidate_df["relative_valuation_TTM_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_TTM_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_TTM_median"] = (np.median([stock_consolidate_df["relative_valuation_TTM_+"][0], stock_consolidate_df["relative_valuation_TTM_-"][0]])).round(2) #这个是根据最新TTM PE估值的价格中位数


0 ACN


In [47]:
import plotly.graph_objects as go


# Create the figure
fig = go.Figure()

# Add stock_price on primary y-axis (left)
fig.add_trace(go.Scatter(
    x=stock_consolidate_df.index,
    y=stock_consolidate_df["stock_price"],
    mode='lines',
    line=dict(color='black'),
    name='Stock Price',
    yaxis="y1"
))

# Add EPS_TTM on secondary y-axis (right)
fig.add_trace(go.Scatter(
    x=stock_consolidate_df.index,
    y=stock_consolidate_df["EPS_TTM"],
    mode='lines',
    fill='tonexty',  # Shadow effect
    line=dict(color='green'),
    name='EPS_TTM',
    yaxis="y2"
))

# Add PE_TTM on secondary y-axis (right)
fig.add_trace(go.Scatter(
    x=stock_consolidate_df.index,
    y=stock_consolidate_df["PE_TTM"],
    mode='lines',
    line=dict(color='blue', dash='dot'),
    name='PE_TTM',
    yaxis="y2"
))

# Update layout to remove grid and configure dual y-axes
fig.update_layout(
    title="Stock Metrics Over Time",
    xaxis=dict(title="Date", showgrid=False),
    yaxis=dict(
        title="Stock Price",
        showgrid=False,
        titlefont=dict(color="black"),
        tickfont=dict(color="black")
    ),
    yaxis2=dict(
        title="EPS_TTM & PE_TTM",
        overlaying="y",  # Overlay with y1
        side="right",
        showgrid=False,
        titlefont=dict(color="blue"),
        tickfont=dict(color="blue")
    ),
    legend_title="Metrics",
    template="plotly_white",
    plot_bgcolor='white'  # Background color
)

# Show the plot
fig.show()

# Financial statement of single stock

In [11]:
stock_financial_dict = {} # the dict will store ticker as key, the string value of 'annual_df' and 'qtr_df' as nested key, actual json as value

ticker_symbols = [
    'UVV'
    ]
# ticker_symbols = electric_utility_symbols
PE_yr_range = 15 # test for 'X' years PE 


# storage process
for j, symbol in enumerate(ticker_symbols):
    print(j, symbol)
    # # COMPANY OVERVIEW
    # # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    # url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    # r = requests.get(url)
    # data = r.json()

    
    # for key, value in data.items():
    #     if key  == 'MarketCapitalization':
    #         stock_mkt_cap = float(value)



    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                # Use pandas index assign instead of direct assign value
                stock_split_record_df = pd.DataFrame({
                'split_factor': [1],
                'effective_date': [datetime.today()]
            })


    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)


    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': 'stock_price'
            }
        ,inplace=True
        )

    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].round(2)

    # modify stock price based on stock split
    for year_i in Monthly_stock_df.index.year:
        for year_j in stock_split_record_df['effective_date'].dt.year:
            if year_i == year_j:

                # stock price to divided the split factor
                Monthly_stock_df.loc[Monthly_stock_df.index.year < year_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.year == year_j].values[0])
    


    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - PE_yr_range) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue
            # clean annualEPS_df
            annualEPS_df = annualEPS_df.sort_values('reportedEPS', ascending=False).drop_duplicates('fiscalDateEnding')
            annualEPS_df = annualEPS_df.sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)

            # calculate PE
            annualEPS_df["PE"] = Monthly_stock_df["stock_price"].values / annualEPS_df['reportedEPS'].values




    # INCOME STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()


    for key, value in data.items():
        if key == 'annualReports':
            annual_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_income_df = annual_income_df.sort_values('fiscalDateEnding', ascending=True)

            # annual measurements cols
            annual_income_metric_cols = annual_income_df.columns[1:]

            # Calculate quarter-over-quarter change percentage
            for column in annual_income_metric_cols:  # Exclude the 'fiscalDateEnding' column
                annual_income_df[column] = pd.to_numeric(annual_income_df[column], errors='coerce')
                annual_income_df[f'{column}_YoY'] = annual_income_df[column].pct_change() * 100 
            
            annual_income_YoY_metric_cols = [col for col in annual_income_df.columns if 'YoY' in col]

            # ratios of income statement calculation
            annual_income_df['gross_margin_%'] = (annual_income_df['grossProfit'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['operating_margin_%'] = (annual_income_df['operatingIncome'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['net_margin_%'] = (annual_income_df['netIncome'] / annual_income_df['totalRevenue']) * 100



        if key == 'quarterlyReports':
            qtr_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_income_df = qtr_income_df.sort_values('fiscalDateEnding', ascending=True)

            # qtr measurements cols
            qtr_income_metric_cols = qtr_income_df.columns[1:]
            
            # Calculate quarter-over-quarter change percentage
            for column in qtr_income_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_income_df[column] = pd.to_numeric(qtr_income_df[column], errors='coerce')
                qtr_income_df[f'{column}_QoQ'] = qtr_income_df[column].pct_change() * 100

            qtr_income_QoQ_metric_cols = [col for col in qtr_income_df.columns if 'QoQ' in col]

            # ratios of income statement calculation
            qtr_income_df['gross_margin_%'] = (qtr_income_df['grossProfit'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['operating_margin_%'] = (qtr_income_df['operatingIncome'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['net_margin_%'] = (qtr_income_df['netIncome'] / qtr_income_df['totalRevenue']) * 100

    income_ratio_cols = [
        'gross_margin_%'
        ,'operating_margin_%'
        ,'net_margin_%'
        ,'netIncome'
        ]
    
    
    # BALANCESHEET
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_balancesheet_df = annual_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_balancesheet_df[column] = pd.to_numeric(annual_balancesheet_df[column], errors='coerce')
                annual_balancesheet_df[f'{column}_YoY'] = annual_balancesheet_df[column].pct_change() * 100

            annual_balancesheet_df['current_ratio'] = (annual_balancesheet_df['totalCurrentAssets'] / annual_balancesheet_df['totalCurrentLiabilities'])
            annual_balancesheet_df['working_capital'] = annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['totalCurrentLiabilities']
            annual_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (annual_balancesheet_df['longTermDebt'] / annual_balancesheet_df['working_capital'])
            annual_balancesheet_df['debtEquity_ratio'] = (annual_balancesheet_df['totalLiabilities'] / annual_balancesheet_df['totalShareholderEquity'])
            annual_balancesheet_df['quick_ratio'] = ((annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['inventory']) / annual_balancesheet_df['totalCurrentLiabilities'])

            
        if key == 'quarterlyReports':
            qtr_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_balancesheet_df = qtr_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_balancesheet_df[column] = pd.to_numeric(qtr_balancesheet_df[column], errors='coerce')
                qtr_balancesheet_df[f'{column}_YoY'] = qtr_balancesheet_df[column].pct_change() * 100

            qtr_balancesheet_df['current_ratio'] = (qtr_balancesheet_df['totalCurrentAssets'] / qtr_balancesheet_df['totalCurrentLiabilities'])
            qtr_balancesheet_df['working_capital'] = qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['totalCurrentLiabilities']
            qtr_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (qtr_balancesheet_df['longTermDebt'] / qtr_balancesheet_df['working_capital'])
            qtr_balancesheet_df['debtEquity_ratio'] = (qtr_balancesheet_df['totalLiabilities'] / qtr_balancesheet_df['totalShareholderEquity'])
            qtr_balancesheet_df['quick_ratio'] = ((qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['inventory']) / qtr_balancesheet_df['totalCurrentLiabilities'])


    balancesheet_ratio_cols = [
        'current_ratio'
        ,'working_capital'
        ,'longTermDebt_to_workingCp_ratio'
        ,'debtEquity_ratio'
        ,'quick_ratio'
        ,'totalShareholderEquity'
    ]


    # CASHFLOW STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_cashflow_df = annual_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_cashflow_df[column] = pd.to_numeric(annual_cashflow_df[column], errors='coerce')
                annual_cashflow_df[f'{column}_YoY'] = annual_cashflow_df[column].pct_change() * 100

            
        if key == 'quarterlyReports':
            qtr_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_cashflow_df = qtr_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_cashflow_df[column] = pd.to_numeric(qtr_cashflow_df[column], errors='coerce')
                qtr_cashflow_df[f'{column}_YoY'] = qtr_cashflow_df[column].pct_change() * 100



    cashflow_ratio_cols = [
        'operatingCashflow'
        ,'capitalExpenditures'
        ,'cashflowFromInvestment'
        ,'cashflowFromFinancing'
        ,'netIncome'
    ]


    ####################
    ### Consolidated ###
    ####################
    stock_ratios_annual_consolidate_df = pd.DataFrame()
    stock_ratios_qtr_consolidate_df = pd.DataFrame()
    stock_PE_annual_consolidate_df = pd.DataFrame()

    stock_ratios_annual_consolidate_df[income_ratio_cols] = annual_income_df[income_ratio_cols]
    stock_ratios_annual_consolidate_df[balancesheet_ratio_cols] = annual_balancesheet_df[balancesheet_ratio_cols]
    stock_ratios_annual_consolidate_df[cashflow_ratio_cols] = annual_cashflow_df[cashflow_ratio_cols]

    stock_ratios_qtr_consolidate_df[income_ratio_cols] = qtr_income_df[income_ratio_cols]
    stock_ratios_qtr_consolidate_df[balancesheet_ratio_cols] = qtr_balancesheet_df[balancesheet_ratio_cols]
    stock_ratios_qtr_consolidate_df[cashflow_ratio_cols] = qtr_cashflow_df[cashflow_ratio_cols]

    # calculating new consolidated metrics 
    # annual df
    stock_ratios_annual_consolidate_df['ROE_%'] = (annual_income_df['netIncome'] / annual_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_annual_consolidate_df['liquidation_value'] = ((annual_balancesheet_df['totalAssets'] - annual_balancesheet_df['intangibleAssets']) - annual_balancesheet_df['totalLiabilities'])
    # stock_ratios_annual_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_annual_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_annual_consolidate_df['fiscalDateEnding'] = annual_balancesheet_df['fiscalDateEnding']

    stock_ratios_annual_consolidate_df['PE'] = annualEPS_df["PE"]
    stock_ratios_annual_consolidate_df['reportedEPS'] = annualEPS_df["reportedEPS"]

    # YoY %
    stock_ratios_annual_consolidate_df['QoQ_gross_margin_%_%'] = stock_ratios_annual_consolidate_df["gross_margin_%"].pct_change() * 100
    stock_ratios_annual_consolidate_df['QoQ_operating_margin_%_%'] = stock_ratios_annual_consolidate_df["operating_margin_%"].pct_change() * 100
    stock_ratios_annual_consolidate_df['QoQ_net_margin_%_%'] = stock_ratios_annual_consolidate_df["net_margin_%"].pct_change() * 100
    stock_ratios_annual_consolidate_df['YoY_reportedEPS_%'] = stock_ratios_annual_consolidate_df["reportedEPS"].pct_change() * 100 # yearly EPS 的增速



    # qtr df
    stock_ratios_qtr_consolidate_df['ROE_%'] = (qtr_income_df['netIncome'] / qtr_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_qtr_consolidate_df['liquidation_value'] = ((qtr_balancesheet_df['totalAssets'] - qtr_balancesheet_df['intangibleAssets']) - qtr_balancesheet_df['totalLiabilities'])
    # stock_ratios_qtr_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_qtr_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_qtr_consolidate_df['fiscalDateEnding'] = qtr_balancesheet_df['fiscalDateEnding']
    
    # QoQ %
    stock_ratios_qtr_consolidate_df['QoQ_gross_margin_%_%'] = stock_ratios_qtr_consolidate_df["gross_margin_%"].pct_change() * 100
    stock_ratios_qtr_consolidate_df['QoQ_operating_margin_%_%'] = stock_ratios_qtr_consolidate_df["operating_margin_%"].pct_change() * 100
    stock_ratios_qtr_consolidate_df['QoQ_net_margin_%_%'] = stock_ratios_qtr_consolidate_df["net_margin_%"].pct_change() * 100




    # annual PE df
    if stock_PE_annual_consolidate_df.empty:
        stock_PE_annual_consolidate_df['fiscalDateEnding'] = annualEPS_df['fiscalDateEnding']

    stock_PE_annual_consolidate_df[f'{symbol}_PE'] = annualEPS_df["PE"].round(2)



    # reformat the columns order
    stock_ratios_annual_consolidate_df = stock_ratios_annual_consolidate_df[['fiscalDateEnding'] + [col for col in stock_ratios_annual_consolidate_df.columns if col != 'fiscalDateEnding']]
    stock_ratios_qtr_consolidate_df = stock_ratios_qtr_consolidate_df[['fiscalDateEnding'] + [col for col in stock_ratios_qtr_consolidate_df.columns if col != 'fiscalDateEnding']]




    # store the stock, dataframe value pair to the dictionary
    # transfer the pandas df to json
    stock_ratios_annual_consolidate_json = stock_ratios_annual_consolidate_df.to_dict()
    stock_ratios_qtr_consolidate_json = stock_ratios_qtr_consolidate_df.to_dict()

    stock_financial_dict[symbol] = {
    'annual': stock_ratios_annual_consolidate_json
    ,'qtr': stock_ratios_qtr_consolidate_json
    }

0 UVV


In [12]:
qtr_balancesheet_df.tail(10)

,fiscalDateEnding,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,accumulatedDepreciationAmortizationPPE,intangibleAssets,intangibleAssetsExcludingGoodwill,goodwill,investments,longTermInvestments,shortTermInvestments,otherCurrentAssets,otherNonCurrentAssets,totalLiabilities,totalCurrentLiabilities,currentAccountsPayable,deferredRevenue,currentDebt,shortTermDebt,totalNonCurrentLiabilities,capitalLeaseObligations,longTermDebt,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding,totalAssets_YoY,totalCurrentAssets_YoY,cashAndCashEquivalentsAtCarryingValue_YoY,cashAndShortTermInvestments_YoY,inventory_YoY,currentNetReceivables_YoY,totalNonCurrentAssets_YoY,propertyPlantEquipment_YoY,accumulatedDepreciationAmortizationPPE_YoY,intangibleAssets_YoY,intangibleAssetsExcludingGoodwill_YoY,goodwill_YoY,investments_YoY,longTermInvestments_YoY,shortTermInvestments_YoY,otherCurrentAssets_YoY,otherNonCurrentAssets_YoY,totalLiabilities_YoY,totalCurrentLiabilities_YoY,currentAccountsPayable_YoY,deferredRevenue_YoY,currentDebt_YoY,shortTermDebt_YoY,totalNonCurrentLiabilities_YoY,capitalLeaseObligations_YoY,longTermDebt_YoY,currentLongTermDebt_YoY,longTermDebtNoncurrent_YoY,shortLongTermDebtTotal_YoY,otherCurrentLiabilities_YoY,otherNonCurrentLiabilities_YoY,totalShareholderEquity_YoY,treasuryStock_YoY,retainedEarnings_YoY,commonStock_YoY,commonStockSharesOutstanding_YoY,current_ratio,working_capital,longTermDebt_to_workingCp_ratio,debtEquity_ratio,quick_ratio
9,2023-09-30,2861772000,2047713000,99683000,99683000,1298508000,424333000,814059000,392926000.0,NaN,74475000,74475000,213856000.0,NaN,70618000.0,NaN,225189000,NaN,1444264000,701145000,70737000,NaN,NaN,311848000,743119000,33273000.0,617086000,301379000.0,NaN,9.517380e+08,147561000.0,15769000.0,1384189000,NaN,1119615000,339241000,25015369,-0.896470,-0.344366,23.802131,23.802131,-0.072646,-5.462181,-2.258576,10.079648,0.0,-3.642127,-3.642127,-0.017298,NaN,-85.161885,0.000000,-0.345180,NaN,-2.049265,-2.330081,-19.946357,NaN,NaN,-15.831635,-1.782825,-2.184266,0.022368,-16.244525,NaN,-5.842722,-1.592541,-32.446558,0.251246,NaN,0.429934,0.235193,0.697193,2.920527,1346568000,0.458266,1.043401,1.068545
8,2023-12-31,2833874000,2013629000,74102000,74102000,1205276000,468415000,820245000,NaN,NaN,71697000,71697000,213891000.0,NaN,75335000.0,NaN,265836000,NaN,1375224000,624874000,89301000,NaN,NaN,375730000,750350000,32453000.0,617225000,365327000.0,NaN,1.015005e+09,134724000.0,26609000.0,1417076000,NaN,1152863000,344467000,25055829,-0.974851,-1.664491,-25.662350,-25.662350,-7.179933,10.388539,0.759896,0.000000,0.0,-3.730111,-3.730111,0.016366,NaN,6.679600,0.000000,18.050171,NaN,-4.780289,-10.878064,26.243691,NaN,NaN,20.484980,0.973061,-2.464461,0.022525,21.218466,NaN,6.647523,-8.699453,68.742469,2.375904,NaN,2.969592,1.540498,0.161741,3.222456,1388755000,0.444445,0.970466,1.293626
7,2024-03-31,2937239000,2102541000,55593000,55593000,1264098000,530647000,834698000,NaN,NaN,68883000,68883000,213869000.0,NaN,76289000.0,NaN,252203000,NaN,1458316000,711358000,108727000,NaN,NaN,427573000,746958000,29658000.0,617364000,417217000.0,NaN,1.064239e+09,150402000.0,27902000.0,1437207000,NaN,1173196000,345596000,25112107,3.647480,4.415511,-24.977733,-24.977733,4.880376,13.285655,1.762035,0.000000,0.0,-3.924850,-3.924850,-0.010286,NaN,1.266344,0.000000,-5.128350,NaN,6.042070,13.840230,21.753396,NaN,NaN,13.797940,-0.452056,-8.612455,0.022520,14.203713,NaN,4.850616,11.637125,4.859258,1.420601,NaN,1.763696,0.327753,0.224610,2.955672,1391183000,0.443769,1.014688,1.178651
6,2024-06-30,3032968000,2198455000,101700000,101700000,1390084000,496932000,834513000,NaN,NaN,66074000,66074000,213810000.0,NaN,75531000.0,-7854000.0,209739000,NaN,1589469000,

In [13]:
qtr_income_df.tail(20)

,fiscalDateEnding,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,grossProfit_QoQ,totalRevenue_QoQ,costOfRevenue_QoQ,costofGoodsAndServicesSold_QoQ,operatingIncome_QoQ,sellingGeneralAndAdministrative_QoQ,researchAndDevelopment_QoQ,operatingExpenses_QoQ,investmentIncomeNet_QoQ,netInterestIncome_QoQ,interestIncome_QoQ,interestExpense_QoQ,nonInterestIncome_QoQ,otherNonOperatingIncome_QoQ,depreciation_QoQ,depreciationAndAmortization_QoQ,incomeBeforeTax_QoQ,incomeTaxExpense_QoQ,interestAndDebtExpense_QoQ,netIncomeFromContinuingOperations_QoQ,comprehensiveIncomeNetOfTax_QoQ,ebit_QoQ,ebitda_QoQ,netIncome_QoQ,gross_margin_%,operating_margin_%,net_margin_%
19,2021-03-31,123980000,617590000,493610000,493610000,62745000,58637000,NaN,61235000,NaN,-5751000.0,4855000.0,5814000,NaN,NaN,NaN,12107000.0,57458000,16734000.0,NaN,40724000,NaN,63272000.0,75379000.0,39361000,-11.125448,-8.223874,-7.465070,-7.465070,4.251819,-1.176371,NaN,-22.794210,NaN,-14.584880,242650.000000,-13.674833,NaN,0.000000,NaN,-1.126991,4.489989,15.026120,NaN,27.978379,NaN,2.507939,1.906204,18.297118,20.074807,10.159653,6.373322
18,2021-06-30,62473000,350029000,287556000,287556000,10605000,49844000,NaN,51868000,NaN,-6135000.0,73000.0,6208000,NaN,121000.0,NaN,12058000.0,5127000,1215000.0,NaN,3912000,NaN,11335000.0,23393000.0,6357000,-49.610421,-43.323402,-41.744292,-41.744292,-83.098255,-14.995651,NaN,-15.296807,NaN,6.677100,-98.496395,6.776746,NaN,278.125000,NaN,-0.404725,-91.076961,-92.739333,NaN,-90.393871,NaN,-82.085283,-68.966158,-83.849496,17.847950,3.029749,1.816135
17,2021-09-30,92683000,453955000,361272000,361272000,29813000,65402000,NaN,62870000,NaN,-6613000.0,517000.0,7130000,NaN,571000.0,NaN,13038000.0,25617000,3862000.0,NaN,17332000,NaN,32747000.0,45785000.0,19510000,48.356890,29.690683,25.635355,25.635355,181.122112,31.213386,NaN,21.211537,NaN,7.791361,608.219178,14.851804,NaN,371.900826,NaN,8.127384,399.648917,217.860082,NaN,343.047035,NaN,188.901632,95.720942,206.905773,20.416781,6.567391,4.297783
16,2021-12-31,131473000,652644000,521171000,521171000,62773000,60267000,NaN,68700000,NaN,-7253000.0,209000.0,7462000,NaN,265000.0,NaN,14014000.0,57660000,13505000.0,NaN,32870000,NaN,65122000.0,79136000.0,34940000,41.852335,43.768435,44.260004,44.260004,110.555798,-7.851442,NaN,9.273103,NaN,9.677907,-59.574468,4.656381,NaN,-53.590193,NaN,7.485811,125.084905,249.689280,NaN,89.649204,NaN,98.864018,72.842634,79.087647,20.144673,9.618260,5.353608
15,2022-03-31,122297000,646973000,524676000,524676000,57124000,65173000,NaN,65173000,NaN,-6829000.0,118000.0,6947000,NaN,2647000.0,NaN,13411000.0,53863000,20081000.0,NaN,47683000,NaN,60810000.0,74221000.0,25770000,-6.979380,-0.868927,0.672524,0.672524,-8.999092,8.140442,NaN,-5.133916,NaN,-5.845857,-43.540670,-6.901635,NaN,898.867925,NaN,-4.302840,-6.585154,48.693077,NaN,45.065409,NaN,-6.621418,-6.210827,-26.244991,18.902953,8.829426,3.983165
14,2022-06-30,79718000,429822000,350104000,350104000,13266000,66452000,NaN,66452000,NaN,-6487000.0,237000.0,6724000,NaN,175000.0,NaN,14129000.0,6164000,3363000.0,NaN,7712000,NaN,12888000.0,27017000.0,6830000,-34.816063,-33.564152,-33.272343,-33.272343,-76.776836,1.962469,NaN,1.962469,NaN,-5.008054,100.847458,-3.210019,NaN,-93.388742,NaN,5.353814,-88.556152,-83.252826,NaN,-83.826521,NaN,-78.806117,-63.599251,-73.496314,18.546747,3.086394,1.589030
13,2022-09-30,110259000,650984000,540725000,540725000,37886000,72373000,NaN,72373000,NaN,-12177000.0,93000.0,12270000,NaN,16000.0,NaN,14165000.0,26048000,6642000.0,NaN,21727000,NaN,38318000.0,52483000.0,21855000,38.311297,51.454323,54.446964,54.446964,185.5

In [15]:
pd.DataFrame(stock_financial_dict['UVV']['qtr']).tail(20)

,fiscalDateEnding,gross_margin_%,operating_margin_%,net_margin_%,netIncome,current_ratio,working_capital,longTermDebt_to_workingCp_ratio,debtEquity_ratio,quick_ratio,totalShareholderEquity,operatingCashflow,capitalExpenditures,cashflowFromInvestment,cashflowFromFinancing,ROE_%,liquidation_value,QoQ_gross_margin_%_%,QoQ_operating_margin_%_%,QoQ_net_margin_%_%
19,2021-03-31,20.074807,10.159653,6.373322,39361000,5.305546,1262201000,0.410530,0.759963,2.622281,1307299000,181791000,32360000,-25666000,-53873000,3.010864,1276121000,-3.161578,13.593616,28.897485
18,2021-06-30,17.847950,3.029749,1.816135,3912000,4.543056,1251294000,0.414209,0.808803,1.670117,1303825000,-126923000,14428000,-12839000,27151000,0.487565,1271651000,-11.092795,-70.178617,-71.504104
17,2021-09-30,20.416781,6.567391,4.297783,21755000,4.150938,1258513000,0.411932,0.842014,1.608852,1297332000,6152000,4217000,961000,9380000,1.503856,1265916000,14.392867,116.763538,136.644427
16,2021-12-31,20.144673,9.618260,5.353608,44155000,3.186126,1197527000,0.433015,0.937904,1.329027,1316004000,69195000,21186000,-117806000,47644000,2.655007,1263452000,-1.332768,46.454816,24.566736
15,2022-03-31,18.902953,8.829426,3.983165,25770000,3.369829,1229287000,0.421827,0.896335,1.409877,1340543000,96458000,13372000,-12977000,-100935000,1.922355,1292198000,-6.164014,-8.201429,-25.598497
14,2022-06-30,18.546747,3.086394,1.589030,2801000,2.607970,1193133000,0.434820,1.070568,0.883836,1325763000,-225771000,15070000,-13610000,245471000,0.515175,1270707000,-1.884390,-65.044228,-60.106347
13,2022-09-30,16.937283,5.819805,3.357225,19406000,2.494083,1192946000,0.434993,1.101453,0.987724,1324854000,-120477000,11518000,-10166000,103717000,1.649616,1268604000,-8.677881,88.563264,111.275157
12,2022-12-31,18.300989,9.751220,5.239995,51361000,3.262790,1353662000,0.455616,1.002476,1.460897,1360792000,162398000,12842000,-10775000,-145896000,3.061452,1317825000,8.051506,67.552342,56.081107
11,2023-03-31,17.696213,7.549796,7.738995,56668000,4.079057,1360903000,0.453235,0.860526,1.733325,1397088000,173293000,15244000,-15799000,-164343000,3.844210,1356851000,-3.304610,-22.575879,47.690899
10,2023-06-30,16.710126,2.131453,-0.398670,-5161000,2.862333,1336917000,0.461471,1.067907,1.052189,1380720000,-103891000,17960000,-17634000,137349000,-0.149487,1335889000,-5.572302,-71.768075,-105.151438


In [16]:
# searching & merge process
YoY_EPS_screener = pd.DataFrame()

for symbol in stock_financial_dict.keys():
    annaul_df = pd.DataFrame(
        stock_financial_dict[symbol]['annual']
    )

    annaul_df = annaul_df[['fiscalDateEnding', 'YoY_reportedEPS_%', 'reportedEPS']].rename(
        columns={
            'YoY_reportedEPS_%': f'{symbol}_YoY_reportedEPS_%'
            ,'reportedEPS': f'{symbol}_reportedEPS'
            }
    )
    annaul_df['fiscalDateEnding'] = pd.to_datetime(annaul_df['fiscalDateEnding']).dt.year


    if YoY_EPS_screener.empty:
        YoY_EPS_screener = annaul_df
    else:
        YoY_EPS_screener = pd.merge(
            YoY_EPS_screener
            ,annaul_df
            ,on='fiscalDateEnding'
            ,how='outer'
        )


In [17]:
YoY_EPS_screener

,fiscalDateEnding,UVV_YoY_reportedEPS_%,UVV_reportedEPS
19,2006,NaN,NaN
18,2007,NaN,NaN
17,2008,NaN,NaN
16,2009,NaN,NaN
15,2010,NaN,NaN
14,2011,NaN,4.1551
13,2012,-21.301533,3.2700
12,2013,80.244648,5.8940
11,2014,-47.958941,3.0673
10,2015,16.379226,3.5697


In [18]:
stock_PE_annual_consolidate_df

,fiscalDateEnding,UVV_PE
0,2025,9.13
1,2024,11.41
2,2023,17.78
3,2022,14.01
4,2021,13.17
5,2020,14.21
6,2019,12.79
7,2018,14.52
8,2017,-58.44
9,2016,16.13


# SP500 sectors

## SP500 sector ticker

In [ ]:
# Function to get the list of S&P 500 companies and their sectors
def get_sp500_companies():
    # Fetch the S&P 500 company symbols and sectors from a reliable source (e.g., Wikipedia)
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tables = pd.read_html(url)
    
    # Extract the relevant table containing the company symbols and sectors
    sp500_df = tables[0]
    
    # Return the DataFrame containing S&P 500 companies and sectors
    return sp500_df[['Symbol', 'GICS Sector', 'GICS Sub-Industry']]


# Function to create a dictionary of sectors and sub-sectors
def create_sector_subsector_dict(df):
    sector_subsector_dict = {}
    for index, row in df.iterrows():
        sector = row['GICS Sector']
        subsector = row['GICS Sub-Industry']
        if sector not in sector_subsector_dict:
            sector_subsector_dict[sector] = [subsector]
        else:
            sector_subsector_dict[sector].append(subsector)
    return sector_subsector_dict

# Function to filter the S&P 500 companies by sector
def company_sector_list(df, sector):
    return df[df['GICS Sector'] == sector]['Symbol'].tolist()

def company_sub_sector_list(df, sub_sector):
    return df[df['GICS Sub-Industry'] == sub_sector]['Symbol'].tolist()


# Get the list of S&P 500 companies and their sectors
sp500_df  = get_sp500_companies()

sp500_companies_sectors = sp500_df ['GICS Sector'].value_counts().index
sp500_companies_sub_sectors = sp500_df ['GICS Sub-Industry'].value_counts().index

sector_subsector_dict = create_sector_subsector_dict(sp500_df)


# Function to create a DataFrame from the sector_subsector_dict
def create_sector_dataframe():
    # Create a list to store dictionacompany_sector_listries representing each row of data
    data = []
    
    # Filter the DataFrame to get stocks in the specified sector
    for sector in sp500_companies_sectors:
        sector_stocks_list = company_sector_list(sp500_df, sector)

        # Iterate over the stocks in the sector and create a dictionary for each
        for i, ticker in enumerate(sector_stocks_list, start=1):
            # Create a dictionary for the current stock in the sector
            row_data = {'Sector': sector, 'Ticker': ticker}
            # Append the dictionary to the list
            data.append(row_data)
    
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df


sector_subsector_dict = create_sector_subsector_dict(sp500_df)


sector_ticker_df = create_sector_dataframe()
# sector_ticker_list = sector_ticker_df[sector_ticker_df['Sector'] == 'Consumer Staples']['Ticker'].tolist()

NameError: name 'pd' is not defined

In [ ]:
# Pivot the DataFrame
sp500_pivot_sector_ticker_df = sector_ticker_df.groupby('Sector')['Ticker'].apply(list).reset_index()

# Transpose to get sectors as columns
sp500_pivot_sector_ticker_df = sp500_pivot_sector_ticker_df.set_index('Sector').T

sp500_pivot_sector_ticker_df

Sector,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
Ticker,"[GOOGL, GOOG, T, CHTR, CMCSA, EA, FOXA, FOX, I...","[ABNB, AMZN, APTV, AZO, BBY, BKNG, BWA, CZR, K...","[MO, ADM, BF.B, BG, CPB, CHD, CLX, KO, CL, CAG...","[APA, BKR, CVX, COP, CTRA, DVN, FANG, EOG, EQT...","[AFL, ALL, AXP, AIG, AMP, AON, APO, ACGL, AJG,...","[ABT, ABBV, A, ALGN, AMGN, BAX, BDX, TECH, BII...","[MMM, AOS, ALLE, AME, ADP, AXON, BA, BR, BLDR,...","[ACN, ADBE, AMD, AKAM, APH, ADI, ANSS, AAPL, A...","[APD, ALB, AMCR, AVY, BALL, CE, CF, CTVA, DOW,...","[ARE, AMT, AVB, BXP, CPT, CBRE, CSGP, CCI, DLR...","[AES, LNT, AEE, AEP, AWK, ATO, CNP, CMS, ED, C..."


In [ ]:
sp500_pivot_sector_ticker_df['Consumer Staples'].iloc[0]

['MO',
 'ADM',
 'BF.B',
 'BG',
 'CPB',
 'CHD',
 'CLX',
 'KO',
 'CL',
 'CAG',
 'STZ',
 'COST',
 'DG',
 'DLTR',
 'EL',
 'GIS',
 'HSY',
 'HRL',
 'K',
 'KVUE',
 'KDP',
 'KMB',
 'KHC',
 'KR',
 'LW',
 'MKC',
 'TAP',
 'MDLZ',
 'MNST',
 'PEP',
 'PM',
 'PG',
 'SJM',
 'SYY',
 'TGT',
 'TSN',
 'WBA',
 'WMT']

In [ ]:
len(sp500_pivot_sector_ticker_df['Consumer Discretionary'].iloc[0])

50

## SP 500 earnings by years

In [ ]:
sp500_eps_df= pd.read_excel('D:\Investment\SP500_sectors\sp500_earnings\sp-500-eps-est.xlsx'
                            ,sheet_name='QUARTERLY DATA')

In [ ]:
sp500_eps_df.columns = [
    'Quarter'
    ,'Ops EPS(Opearting)'
    ,'Reported EPS(GAAP)'
    ,'Div/share'
    ,'Sales/share'
    ,'BVPS'
    ,'CapEx/share'
    ,'Price'
    ,'Divisor'
]

In [ ]:
sp500_eps_df = sp500_eps_df[5::].fillna(0)
sp500_eps_df_no_first = sp500_eps_df \
    .iloc[1:] \
    .sort_values(['Quarter'], ascending=True) \
    .copy()

#####################
# Column enrichment #
#####################
sp500_eps_df['Year'] = pd.to_datetime(sp500_eps_df['Quarter']).dt.year
sp500_eps_df["Month"] = pd.to_datetime(sp500_eps_df['Quarter']).dt.month

# Calculate year end EPS
last_quarter_prices_df = sp500_eps_df[sp500_eps_df["Month"] == 12][["Year", "Price"]]
price_map = dict(zip(last_quarter_prices_df["Year"], last_quarter_prices_df["Price"]))
sp500_eps_df["Yearly price"] = sp500_eps_df["Year"].map(price_map)


# Calculate rolling 4-quarter sum
sp500_eps_df_no_first["Ops EPS(Opearting) TTM"] = sp500_eps_df_no_first["Ops EPS(Opearting)"].rolling(window=4).sum()
sp500_eps_df_no_first["Reported EPS(GAAP) TTM"] = sp500_eps_df_no_first["Reported EPS(GAAP)"].rolling(window=4).sum()
sp500_eps_df_no_first["Div/share TTM"] = sp500_eps_df_no_first["Div/share"].rolling(window=4).sum()


# Merge back with original dataframe
sp500_eps_df = sp500_eps_df.merge(
    sp500_eps_df_no_first[
        [
            "Quarter"
            ,"Ops EPS(Opearting) TTM"
            ,'Reported EPS(GAAP) TTM'
            ,'Div/share TTM'
            ]
        ]
    ,on="Quarter"
    ,how="left"
    )


######################
# Metrics calcuation #
######################
sp500_eps_df['PE TTM Ops'] = sp500_eps_df['Price'] / sp500_eps_df['Ops EPS(Opearting) TTM']
sp500_eps_df['PE TTM GAAP'] = sp500_eps_df['Price'] / sp500_eps_df['Reported EPS(GAAP) TTM']
sp500_eps_df['Earnings yield(Ops)'] = round((sp500_eps_df['Ops EPS(Opearting) TTM'] / sp500_eps_df['Price']) * 100, 2)
sp500_eps_df['Earnings yield(GAAP)'] = round((sp500_eps_df['Reported EPS(GAAP) TTM'] / sp500_eps_df['Price']) * 100, 2)
sp500_eps_df['Div payout ratio(Opearting)'] = round((sp500_eps_df['Div/share TTM'] / sp500_eps_df['Ops EPS(Opearting) TTM']) * 100 ,2)
sp500_eps_df['Div payout ratio(GAAP)'] = round((sp500_eps_df['Div/share TTM'] / sp500_eps_df['Reported EPS(GAAP) TTM']) * 100, 2)
# sp500_eps_df["Ops EPS(Opearting) TTM Growth Rate"] = round(((sp500_eps_df["Ops EPS(Opearting) TTM"] - sp500_eps_df["Ops EPS(Opearting) TTM"].shift(-1)) 
#                                                        / sp500_eps_df["Ops EPS(Opearting) TTM"])*100, 2)
# sp500_eps_df["Reported EPS(GAAP) TTM Growth Rate"] = round(((sp500_eps_df["Reported EPS(GAAP) TTM"] - sp500_eps_df["Reported EPS(GAAP) TTM"].shift(-1)) 
#                                                        / sp500_eps_df["Reported EPS(GAAP) TTM"])*100, 2)


# 1.section: YoY growth rate
sp500_eps_df_yrly_ops_eps = sp500_eps_df.groupby(['Year'])['Ops EPS(Opearting)'].sum().reset_index()
sp500_eps_df_yrly_gaap_eps = sp500_eps_df.groupby(['Year'])['Reported EPS(GAAP)'].sum().reset_index()
sp500_eps_df_yrly_ops_eps['YoY growth rate(Ops)'] = sp500_eps_df_yrly_ops_eps['Ops EPS(Opearting)'].pct_change()
sp500_eps_df_yrly_gaap_eps['YoY growth rate(GAAP)'] = sp500_eps_df_yrly_gaap_eps['Reported EPS(GAAP)'].pct_change()

# Create mapping dictionary: {year: YoY growth}
growth_map_ops_eps = dict(zip(sp500_eps_df_yrly_ops_eps["Year"], sp500_eps_df_yrly_ops_eps["YoY growth rate(Ops)"]))
growth_map_gaap_eps = dict(zip(sp500_eps_df_yrly_gaap_eps["Year"], sp500_eps_df_yrly_gaap_eps["YoY growth rate(GAAP)"]))
yrly_map_ops_eps = dict(zip(sp500_eps_df_yrly_ops_eps["Year"], sp500_eps_df_yrly_ops_eps["Ops EPS(Opearting)"]))
yrly_map_gaap_eps = dict(zip(sp500_eps_df_yrly_gaap_eps["Year"], sp500_eps_df_yrly_gaap_eps["Reported EPS(GAAP)"]))

# Map back to original df
sp500_eps_df["YoY growth rate(Ops)"] = round(sp500_eps_df["Year"].map(growth_map_ops_eps)*100, 2)
sp500_eps_df["YoY growth rate(GAAP)"] = round(sp500_eps_df["Year"].map(growth_map_gaap_eps)*100, 2)
sp500_eps_df["Yearly EPS (Ops)"] = sp500_eps_df["Year"].map(yrly_map_ops_eps)
sp500_eps_df["Yearly EPS (GAAP)"] = sp500_eps_df["Year"].map(yrly_map_gaap_eps)


# 2.section: PE TTM AVG Section
PE_TTM_ops_avg = sp500_eps_df.loc[sp500_eps_df["PE TTM Ops"] != 0, "PE TTM Ops"].mean(skipna=True)
PE_TTM_gaap_avg = sp500_eps_df.loc[sp500_eps_df["PE TTM GAAP"] != 0, "PE TTM GAAP"].mean(skipna=True)
sp500_eps_df['PE TTM Ops AVG'] = round(PE_TTM_ops_avg, 2)
sp500_eps_df['PE TTM GAAP AVG'] = round(PE_TTM_gaap_avg, 2)

# 3. section: PE 5 years rolling average
sp500_eps_df['PE Yrly Ops'] = sp500_eps_df['Yearly price'] / sp500_eps_df["Yearly EPS (Ops)"]
sp500_eps_df['PE Yrly GAAP'] = sp500_eps_df["Yearly price"] / sp500_eps_df["Yearly EPS (GAAP)"]

sp500_eps_df_yrly_ops_PE = sp500_eps_df[sp500_eps_df["PE Yrly Ops"] != 0] \
                            .groupby("Year", as_index=False)["PE Yrly Ops"] \
                            .mean()
sp500_eps_df_yrly_gaap_PE = sp500_eps_df[sp500_eps_df["PE Yrly GAAP"] != 0] \
                            .groupby("Year", as_index=False)["PE Yrly GAAP"] \
                            .mean()

sp500_eps_df_yrly_ops_PE["PE Ops 5Y Avg"] = sp500_eps_df_yrly_ops_PE["PE Yrly Ops"].rolling(window=5, min_periods=1).mean()
sp500_eps_df_yrly_ops_PE["PE Ops 10Y Avg"] = sp500_eps_df_yrly_ops_PE["PE Yrly Ops"].rolling(window=10, min_periods=1).mean()
sp500_eps_df_yrly_gaap_PE["PE GAAP 5Y Avg"] = sp500_eps_df_yrly_gaap_PE["PE Yrly GAAP"].rolling(window=5, min_periods=1).mean()
sp500_eps_df_yrly_gaap_PE["PE GAAP 10Y Avg"] = sp500_eps_df_yrly_gaap_PE["PE Yrly GAAP"].rolling(window=10, min_periods=1).mean()


# Create mapping dictionaries
PE5_ops_map = dict(zip(sp500_eps_df_yrly_ops_PE["Year"], sp500_eps_df_yrly_ops_PE["PE Ops 5Y Avg"]))
PE10_ops_map = dict(zip(sp500_eps_df_yrly_ops_PE["Year"], sp500_eps_df_yrly_ops_PE["PE Ops 10Y Avg"]))
PE5_gaap_map = dict(zip(sp500_eps_df_yrly_gaap_PE["Year"], sp500_eps_df_yrly_gaap_PE["PE GAAP 5Y Avg"]))
PE10_gaap_map = dict(zip(sp500_eps_df_yrly_gaap_PE["Year"], sp500_eps_df_yrly_gaap_PE["PE GAAP 10Y Avg"]))


# Map to quarterly dataframe
sp500_eps_df["PE Ops 5Y Avg"] = sp500_eps_df["Year"].map(PE5_ops_map)
sp500_eps_df["PE Ops 10Y Avg"] = sp500_eps_df["Year"].map(PE10_ops_map)
sp500_eps_df["PE GAAP 5Y Avg"] = sp500_eps_df["Year"].map(PE5_gaap_map)
sp500_eps_df["PE GAAP 10Y Avg"] = sp500_eps_df["Year"].map(PE10_gaap_map)

In [ ]:
sp500_eps_df.head(10)

,Quarter,Ops EPS(Opearting),Reported EPS(GAAP),Div/share,Sales/share,BVPS,CapEx/share,Price,Divisor,Year,Month,Yearly price,Ops EPS(Opearting) TTM,Reported EPS(GAAP) TTM,Div/share TTM,PE TTM Ops,PE TTM GAAP,Earnings yield(Ops),Earnings yield(GAAP),Div payout ratio(Opearting),Div payout ratio(GAAP),YoY growth rate(Ops),YoY growth rate(GAAP),Yearly EPS (Ops),Yearly EPS (GAAP),PE TTM Ops AVG,PE TTM GAAP AVG,PE Yrly Ops,PE Yrly GAAP,PE Ops 5Y Avg,PE Ops 10Y Avg,PE GAAP 5Y Avg,PE GAAP 10Y Avg
0,2025-06-30,0.00,0.00,19.484935,0.000000,0.000000,0.000000,6204.953952,8461.081057,2025,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-75.36,-74.36,57.51,53.89,19.46,24.42,NaN,NaN,22.482021,22.251477,24.771408,25.455504
1,2025-03-31,57.51,53.89,19.372895,488.420003,1197.110200,30.911700,5611.852607,8473.831556,2025,3,NaN,236.24,216.69,76.145301,23.754879,25.898069,4.21,3.86,32.23,35.14,-75.36,-74.36,57.51,53.89,19.46,24.42,NaN,NaN,22.482021,22.251477,24.771408,25.455504
2,2024-12-31,61.21,57.69,19.810500,508.588989,1178.567800,34.471600,5881.627648,8467.894518,2024,12,5881.627648,233.36,210.17,74.832255,25.204095,27.985096,3.97,3.57,32.07,35.61,9.29,9.22,233.36,210.17,19.46,24.42,25.204095,27.985096,24.124493,22.061110,27.797731,25.272067
3,2024-09-30,59.16,51.99,18.680400,501.354000,1166.346100,30.891700,5762.484883,8451.342154,2024,9,5881.627648,226.05,200.27,73.400300,25.492081,28.773580,3.92,3.48,32.47,36.65,9.29,9.22,233.36,210.17,19.46,24.42,25.204095,27.985096,24.124493,22.061110,27.797731,25.272067
4,2024-06-28,58.36,53.12,18.281506,488.705399,1139.314000,28.457100,5460.482621,8395.388376,2024,6,5881.627648,219.14,195.93,71.975800,24.917781,27.869559,4.01,3.59,32.84,36.74,9.29,9.22,233.36,210.17,19.46,24.42,25.204095,27.985096,24.124493,22.061110,27.797731,25.272067
5,2024-03-28,54.63,47.37,18.059849,471.949169,1116.727900,27.060700,5254.354401,8388.772132,2024,3,5881.627648,215.62,191.39,70.824826,24.368585,27.453652,4.10,3.64,32.85,37.01,9.29,9.22,233.36,210.17,19.46,24.42,25.204095,27.985096,24.124493,22.061110,27.797731,25.272067
6,2023-12-29,53.90,47.79,18.378545,490.201506,1106.214712,30.133461,4769.829411,8394.128747,2023,12,4769.829411,213.53,192.43,70.303692,22.337983,24.787348,4.48,4.03,32.92,36.53,8.42,11.39,213.53,192.43,19.46,24.42,22.337983,24.787348,23.196177,21.362416,26.833653,24.485973
7,2023-09-29,52.25,47.65,17.255900,468.524082,1076.237542,27.013050,4288.054123,8380.848066,2023,9,4769.829411,210.00,184.25,69.313136,20.419305,23.273021,4.90,4.30,33.01,37.62,8.42,11.39,213.53,192.43,19.46,24.42,22.337983,24.787348,23.196177,21.362416,26.833653,24.485973
8,2023-06-30,54.84,48.58,17.130532,462.132679,1061.913100,25.961900,4450.381312,8350.369620,2023,6,4769.829411,208.10,181.01,68.714936,21.385782,24.586384,4.68,4.07,33.02,37.96,8.42,11.39,213.53,192.43,19.46,24.42,22.337983,24.787348,23.196177,21.362416,26.833653,24.485973
9,2023-03-31,52.54,48.41,17.538715,451.437952,1045.266400,25.460100,4109.312445,8357.127040,2023,3,4769.829411,200.13,175.17,68.211442,20.533216,23.458997,4.87,4.26,34.08,38.94,8.42,11.39,213.53,192.43,19.46,24.42,22.337983,24.787348,23.196177,21.362416,26.833653,24.485973


In [ ]:
sp500_eps_df.to_excel('D:\Investment\SP500_sectors\sp500_earnings\sp-500-eps-est-hist.xlsx')

# Stock PE by years by tickers

In [ ]:
stock_financial_dict = {} # the dict will store ticker as key, the string value of 'annual_df' and 'qtr_df' as nested key, actual json as value
stock_PE_annual_consolidate_df = pd.DataFrame()

ticker_symbols = [
'MNST'
,'KO'
,'KDP'
,'PEP'
 ]
# ticker_symbols = electric_utility_symbols
PE_yr_range = 10 # test for 'X' years PE 

In [ ]:

# storage process
for j, symbol in enumerate(ticker_symbols):
    print(j, symbol)
    # # COMPANY OVERVIEW
    # # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    # url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    # r = requests.get(url)
    # data = r.json()


    # for key, value in data.items():
    #     if key  == 'MarketCapitalization':
    #         stock_mkt_cap = float(value)



    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                # Use pandas index assign instead of direct assign value
                stock_split_record_df = pd.DataFrame({
                'split_factor': [1],
                'effective_date': [datetime.today()]
            })



    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)


    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': 'stock_price'
            }
        ,inplace=True
        )

    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].round(2)

    # modify stock price based on stock split
    for year_i in Monthly_stock_df.index.year:
        for year_j in stock_split_record_df['effective_date'].dt.year:
            if year_i == year_j:

                # stock price to divided the split factor
                Monthly_stock_df.loc[Monthly_stock_df.index.year < year_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.year == year_j].values[0])
    


    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - PE_yr_range) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue
            # clean annualEPS_df
            annualEPS_df = annualEPS_df.sort_values('reportedEPS', ascending=False).drop_duplicates('fiscalDateEnding')
            annualEPS_df = annualEPS_df.sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)

            # calculate PE
            annualEPS_df["PE"] = Monthly_stock_df["stock_price"].values / annualEPS_df['reportedEPS'].values


    ####################
    ### Consolidated ###
    ####################

    # annual PE df
    if stock_PE_annual_consolidate_df.empty:
        stock_PE_annual_consolidate_df['fiscalDateEnding'] = annualEPS_df['fiscalDateEnding']

    stock_PE_annual_consolidate_df[f'{symbol}_PE'] = annualEPS_df["PE"].round(2)


stock_PE_annual_consolidate_df['industry_avg_PE'] = stock_PE_annual_consolidate_df.iloc[:, 1:].sum(axis=1) / (stock_PE_annual_consolidate_df.shape[1] -1)  # Sum by row

0 MNST
1 KO
2 KDP
3 PEP


In [ ]:
stock_PE_annual_consolidate_df

,fiscalDateEnding,MNST_PE,KO_PE,KDP_PE,PEP_PE,industry_avg_PE
0,2024,43.44,26.72,23.97,24.53,29.6650
1,2023,36.46,21.91,18.61,22.29,24.8175
2,2022,22.76,25.65,21.23,26.61,24.0625
3,2021,18.68,25.41,23.04,27.79,23.7300
4,2020,17.25,28.12,22.86,26.87,23.7750
5,2019,15.35,26.23,23.73,24.76,22.5175
6,2018,13.67,22.66,8.90,19.55,16.1950
7,2017,22.77,24.02,22.11,23.20,23.0250
8,2016,17.05,21.71,20.61,29.98,22.3375
9,2015,25.08,21.48,23.18,21.91,22.9125


## Appendix

In [ ]:
# Consumer Staples

ticker_symbols = [
    'MO',
    'ADM',
    # 'BF.B', # DATA ISSUE
    'BG',
    'CPB',
    'CHD',
    'CLX',
    'KO',
    'CL',
    'CAG',
    'STZ', # !!
    'COST',
    'DG',
    'DLTR', # !!
    'EL',
    'GIS',
    'HSY',
    'HRL',
    'K',
    'KVUE',
    'KDP',
    'KMB',
    'KHC',
    'KR', # DATA ISSUE
    'LW',
    'MKC', # DATA ISSUE
    'TAP',
    'MDLZ',
    'MNST',
    'PEP',
    'PM',
    'PG',
    'SJM',
    'SYY',
    'TGT', # !!
    'TSN', # !!
    'WBA', # DATA ISSUE
    'WMT'
]

In [ ]:
# Energy

ticker_symbols = [
'APA',
 'BKR', # !! HIGH PE, LOW PROFITABILITY, HIGH FORECAST,  BUT CANNOT BEAT FORECAST, 总结与TRGP类似, 基本面更稳妥, 盈利预期高, 有待考证
 'CVX',
 'COP',
 'CTRA',
 'DVN',
 'FANG',
 'EOG',
 'EQT',
 'XOM',
 'HAL', # !!!! 与SLB类似
 'HES', # !  HIGH PE, UNDERVALUE, 市场炒作高预期, 2024表现强劲, 已经被超热了
 'KMI',
 'MRO', # PROFITABILITY, STEADY GROWTH FORECAST, AND CAN BEAT FORECAST, 总结基本面良好，与HAL类型相似, 市场炒作热度比HAL更高, 值得研究
 'TRGP', # !！ HGIH PE
 'MPC',
 'OXY',
 'OKE', # !!
 'PSX',
 'SLB', # !!! A BIT HIGH PE, GOOD ROE, MED, HIGH ROE, RELATIVE MORE DEBT THAN EQUITY, LOW PROFITABILITY, HIGH FORECAST, BUT CANNOT BEAT FORECAST, 总结基本面一般, 但是盈利预期非常高, 有待考证
 'VLO',
 'WMB'
 ]

In [ ]:
electric_utility_symbols = [
'CEPU'
,'OTTR'
,'PAM'
,'SO' ##
,'DUK'
,'NRG'
,'IDA'
,'DTE' #
,'CEG' ##
,'SRE'
,'ETR'
,'PEG'
,'MGEE'
,'LNT'
,'BKH'
]